In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import f1_score, ConfusionMatrixDisplay
import pandas as pd
import os
import pickle
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## Training Data

In [3]:
file_path = r'.\cifar-100-python\cifar-100-python\train'

with open(file_path, 'rb') as f:
    data_dict = pickle.load(f, encoding='latin1')


print(data_dict.keys())  


dict_keys(['filenames', 'batch_label', 'fine_labels', 'coarse_labels', 'data'])


In [4]:
data = pd.DataFrame({
    'image_data': list(data_dict['data']),  # Each row will contain 3072 values for image data
    'fine_label': data_dict['fine_labels']  # Fine-grained labels
})

In [5]:
data['image_data'] = data['image_data'].apply(
    lambda x: np.array(x).reshape((3, 32, 32)).transpose(1, 2, 0)
)

In [ ]:
df = pd.DataFrame(data)

# Data Augmentation
def augment_image(image_array):
    # Convert to tensor
    image = tf.convert_to_tensor(image_array, dtype=tf.float32)
    image = tf.image.random_flip_left_right(image)  
    image = tf.image.random_flip_up_down(image)    
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.image.random_hue(image, max_delta=0.02)
    return image.numpy()

# Function to create multiple augmented images for each row
def create_augmented_images(image_array, fine_label, num_augments=3):
    augmented_images = []
    for _ in range(num_augments):
        augmented_images.append({"image_data": augment_image(image_array), "fine_label": fine_label})
    return augmented_images

# Augmented data for the entire DataFrame
augmented_data = []
for _, row in df.iterrows():
    augmented_data.extend(create_augmented_images(row["image_data"], row["fine_label"], num_augments=3))


augmented_df = pd.DataFrame(augmented_data)
print(augmented_df.head())

                                          image_data  fine_label
0  [[[248.94589, 248.60504, 246.94533], [248.9458...          19
1  [[[94.06731, 124.79763, 39.887543], [95.158936...          19
2  [[[99.19211, 123.30453, 58.27069], [100.060715...          19
3  [[[251.21516, 251.05232, 250.35997], [249.3599...          29
4  [[[245.12282, 244.61502, 243.25955], [243.4983...          29


In [8]:
merged_df = pd.concat([df, augmented_df], ignore_index=True)

In [9]:
merged_df['image_data'] = merged_df['image_data'] / 255.0
merged_df['image_data'] = merged_df['image_data'].map(lambda x: (np.round(x, 2)))

In [10]:
train = merged_df.groupby('fine_label').apply(lambda x: x.sample(2000)).reset_index(drop=True)

In [11]:
images = np.array(train['image_data'].tolist()) 
labels = np.array(train['fine_label'])  

In [12]:
valid = train.groupby('fine_label').apply(lambda x: x.sample(200)).reset_index(drop=True)

In [13]:
images_valid = np.array(valid['image_data'].tolist())
labels_valid = np.array(valid['fine_label'])

In [ ]:
input_shape = (71, 71, 3)


def preprocess_input_xception_batch(images, batch_size):
    processed_images = []
    for i in range(0, len(images), batch_size):
        batch = images[i:i + batch_size]
        batch_resized = [tf.image.resize(image, (71, 71)) for image in batch]  # Resize to 71x71
        batch_preprocessed = tf.keras.applications.xception.preprocess_input(np.array(batch_resized))
        processed_images.extend(batch_preprocessed)
    return np.array(processed_images)

# Loading the base Xception model
base_model = Xception(
    weights="imagenet",  # Using pre-trained ImageNet weights
    include_top=False,   # Exclude fully connected layers
    input_tensor=Input(shape=input_shape),
)

# Add custom layers on top of Xception
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling
x = Dense(512, activation="relu")(x)  # Dense layer with 512 neurons
x = tf.keras.layers.Dropout(0.5)(x)  # Dropout for regularization
output = Dense(100, activation="softmax")(x)  # CIFAR-100 has 100 classes

# Define the complete model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False


model.compile(
    optimizer=Adam(learning_rate=0.0002),
    loss="sparse_categorical_crossentropy", 
    metrics=["sparse_categorical_accuracy"],
)


model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 71, 71, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 35, 35, 32)   864         ['input_1[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 35, 35, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                                  
 block1_conv1_act (Activation)  (None, 35, 35, 32)   0           ['block1_conv1_bn[0][0]']    

In [15]:
x_train = preprocess_input_xception_batch(train['image_data'].tolist(), batch_size=10000)
x_test = preprocess_input_xception_batch(valid['image_data'].tolist(), batch_size=10000)

In [ ]:

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(x_train)

In [17]:
# Train the model
history = model.fit(
    datagen.flow(x_train, labels, batch_size=32),
    validation_data=(x_test, labels_valid),
    epochs=20,
    steps_per_epoch=x_train.shape[0] // 32,
    verbose=1,
)

Epoch 1/20
6250/6250 [==============================] - 217s 33ms/step - loss: 4.5456 - sparse_categorical_accuracy: 0.0239 - val_loss: 4.4504 - val_sparse_categorical_accuracy: 0.0349
Epoch 2/20
6250/6250 [==============================] - 206s 33ms/step - loss: 4.4082 - sparse_categorical_accuracy: 0.0396 - val_loss: 4.3324 - val_sparse_categorical_accuracy: 0.0519
Epoch 3/20
6250/6250 [==============================] - 159s 25ms/step - loss: 4.3276 - sparse_categorical_accuracy: 0.0488 - val_loss: 4.2631 - val_sparse_categorical_accuracy: 0.0573
Epoch 4/20
6250/6250 [==============================] - 207s 33ms/step - loss: 4.2780 - sparse_categorical_accuracy: 0.0541 - val_loss: 4.2187 - val_sparse_categorical_accuracy: 0.0631
Epoch 5/20
6250/6250 [==============================] - 185s 30ms/step - loss: 4.2425 - sparse_categorical_accuracy: 0.0593 - val_loss: 4.1849 - val_sparse_categorical_accuracy: 0.0694
Epoch 6/20
6250/6250 [==============================] - 183s 29ms/step - lo

In [47]:
for layer in base_model.layers:
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.000005),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [ ]:

# Continue training
history_finetune = model.fit(
    datagen.flow(x_train, labels, batch_size=32),
    validation_data=(x_test, labels_valid),
    epochs=25,
    steps_per_epoch=x_train.shape[0] // 32,
    verbose=1,
)


Epoch 1/5
6250/6250 [==============================] - 307s 49ms/step - loss: 0.8543 - sparse_categorical_accuracy: 0.7482 - val_loss: 3.6751 - val_sparse_categorical_accuracy: 0.8388
Epoch 2/5
6250/6250 [==============================] - 293s 47ms/step - loss: 0.8076 - sparse_categorical_accuracy: 0.7611 - val_loss: 1.6136 - val_sparse_categorical_accuracy: 0.8576
Epoch 3/5
6250/6250 [==============================] - 293s 47ms/step - loss: 0.7692 - sparse_categorical_accuracy: 0.7715 - val_loss: 2.8508 - val_sparse_categorical_accuracy: 0.8627
Epoch 4/5
6250/6250 [==============================] - 296s 47ms/step - loss: 0.7301 - sparse_categorical_accuracy: 0.7824 - val_loss: 1.7991 - val_sparse_categorical_accuracy: 0.8780
Epoch 5/5
6250/6250 [==============================] - 294s 47ms/step - loss: 0.6949 - sparse_categorical_accuracy: 0.7924 - val_loss: 0.8663 - val_sparse_categorical_accuracy: 0.8884


In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.000002),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [ ]:
# Continue training
history_finetune = model.fit(
    datagen.flow(x_train, labels, batch_size=32),
    validation_data=(x_test, labels_valid),
    epochs=10,
    steps_per_epoch=x_train.shape[0] // 32,
    verbose=1,
)

6250/6250 [==============================] - 310s 50ms/step - loss: 0.6050 - sparse_categorical_accuracy: 0.8171 - val_loss: 1.1567 - val_sparse_categorical_accuracy: 0.9061


## Test Data

In [ ]:
file_path = r'.\cifar-100-python\cifar-100-python\test'
with open(file_path, 'rb') as f:
    test_dict = pickle.load(f, encoding='latin1')


print(test_dict.keys()) 

dict_keys(['filenames', 'batch_label', 'fine_labels', 'coarse_labels', 'data'])


In [22]:
test = pd.DataFrame({
    'image_data': list(test_dict['data']),  
    'label': test_dict['fine_labels']
})

In [23]:
test['image_data'] = test['image_data'].apply(
    lambda x: np.array(x).reshape((3, 32, 32)).transpose(1, 2, 0)
)

In [24]:
test['image_data'] = test['image_data'] / 255.0

In [25]:
test['image_data'] = test['image_data'].map(lambda x: (np.round(x, 2)))

In [26]:
test_images = np.array(test['image_data'].tolist()) 
test_labels = np.array(test['label'])  

In [27]:
test_images.shape

(10000, 32, 32, 3)

In [28]:
test = preprocess_input_xception_batch(test['image_data'].tolist(), batch_size=1000)

## Prediction

In [52]:
test_loss, test_accuracy = model.evaluate(test, test_labels, verbose=1)
print(f"Test accuracy: {test_accuracy:.2f}")

313/313 [==============================] - 4s 14ms/step - loss: 2.3768 - sparse_categorical_accuracy: 0.7293
Test accuracy: 0.73


In [40]:
model.save(os.path.join('models','xception_3.h5'))